# Training and Testing A Visual Transformer Model
Here we test a visual Transfrom ViT from [An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale](https://arxiv.org/abs/2010.11929)

In [5]:
!pip install tensorboard


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import torch
print(torch.__version__)
import torchvision as tv
import torchvision.transforms.v2 as v2
from our_datasets import Country_images
#from Country_dict import comp_country_dict
from ViT import VisionTransformer

USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
#get models
print(f"Using {device} device")

2.2.2
Using cpu device


## Load our Dataset and Create our dataloaders

In [7]:
batch_size = 64
weights = tv.models.ViT_B_32_Weights.DEFAULT
transform = v2.Compose([weights.transforms(), ])


dataset_path = "data/compressed_dataset/"
dataset = Country_images("country.csv",dataset_path,transform=transform)
num_classes = dataset.get_num_classes()
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset,lengths=[0.7,0.1,0.2])
data_loader_params = {
    'batch_size': batch_size,  # Batch size for data loading
    #'num_workers': 10,  # Number of subprocesses to use for data loading
    #'persistent_workers': True,  # If True, the data loader will not shutdown the worker processes after a dataset has been consumed once. This allows to maintain the worker dataset instances alive.
    #'pin_memory': True,  # If True, the data loader will copy Tensors into CUDA pinned memory before returning them. Useful when using GPU.
    #'pin_memory_device': 'cuda' ,  # Specifies the device where the data should be loaded. Commonly set to use the GPU.
}
train_dataloader      = torch.utils.data.DataLoader(train_dataset, **data_loader_params, shuffle=True)
val_dataloader        = torch.utils.data.DataLoader(val_dataset, **data_loader_params, shuffle=True)
test_dataloader       = torch.utils.data.DataLoader(test_dataset, **data_loader_params, shuffle=False)

## Load our model

In [11]:
#hyperparameters:
lr = 0.001
#maximum learning rate we will let our model train in order to train faster at the start
max_lr = 0.01
weight_decay = 0.000001
EPOCHS = 40
#end hyperparameters

#model and optimizers
#model = tv.models.vit_b_16(weights=weights)
model = tv.models.vit_b_32(weights=weights)
#model = tv.models.vit_b_16(weights=weights)
#model = tv.models.DenseNet()
model.device = device
model.name = "ViT_B_32"
model.path = "Trained_Models/ViT/" #where to save our best model
print(model.path)
#redfine our output layer to output our classes
model.heads.head = torch.nn.Linear(in_features=model.heads.head.in_features,out_features=num_classes,bias=True)
print(model)
#loss_weights = torch.ones(num_classes,device=device)s
#loss_weights[21] = 0.25
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
#scales the gradients, neccessary for mixed precision data types to properly converge
scaler = torch.amp.GradScaler(device=device) if torch.cuda.is_available() else None
use_amp = torch.cuda.is_available()
#change our learning rate based on far we are in training and if we are improving
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, total_steps=EPOCHS*len(train_dataloader))

#added data to our model for ease of use (and to prevent passing so many variables to our training function)


Trained_Models/ViT/
VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=T

## Train our Model

In [ ]:
#we will call the function we defined in "Training_Functions.py"
from Training_Functions import TrainModel
model_train = True
fine_tune = False

if model_train and fine_tune:
    print("Starting Fine Tuning")
    checkpoint = torch.load(model.path+model.name+"-best.pth")
    model.load_state_dict(checkpoint)
    model = model.to(device)
    TrainModel(model,EPOCHS, loss_fn, train_dataloader, val_dataloader, optimizer, lr_scheduler, use_amp,scaler )
elif model_train:
    print("Starting Training")
    model = model.to(device)
    TrainModel(model,EPOCHS, loss_fn, train_dataloader, val_dataloader, optimizer, lr_scheduler,use_amp, scaler)
else:
    print("Loading Model")
    checkpoint = torch.load(model.path+model.name+"-best.pth")
    model.load_state_dict(checkpoint)
    model = model.to(device)

Starting Training
EPOCH 1:


## Test our Model

In [ ]:
#we will call the function we defined in "Training_Functions.py"
from Training_Functions import TestModel
TestModel(model, test_dataloader, loss_fn)

RuntimeError: image::decode_image() Expected a value of type 'Tensor' for argument '_0' but instead found type 'str'.
Position: 0
Value: 'data/compressed_dataset/United Kingdom/canvas_1629872610.jpg'
Declaration: image::decode_image(Tensor _0, int _1) -> Tensor _0
Cast error details: Unable to cast data/compressed_dataset/United Kingdom/canvas_1629872610.jpg to Tensor